In [1]:
import pandas as pd
import sqlite3

Создадим подключение к базе данных с помощью библиотеки sqlite3

In [2]:
col_names = ['datetime', 'user']
connect = sqlite3.connect('../data/checking-logs.sqlite')

Получим схему просмотров страниц таблицы

In [3]:
query = 'PRAGMA table_info(checker);'
pd.io.sql.read_sql(query, connect)

,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,status,TEXT,0,None,0
2,2,success,INTEGER,0,None,0
3,3,timestamp,TIMESTAMP,0,None,0
4,4,numTrials,INTEGER,0,None,0
5,5,labname,TEXT,0,None,0
6,6,uid,TEXT,0,None,0


Получим первые 10 строк просмотров страниц таблицы, чтобы проверить, как выглядит таблица

In [4]:
query = 'SELECT * FROM checker LIMIT 10;'
pd.io.sql.read_sql(query, connect)

,index,status,success,timestamp,numTrials,labname,uid
0,0,checking,0,2020-04-16 21:12:50.740474,5,None,admin_1
1,1,ready,0,2020-04-16 21:12:54.708365,5,code_rvw,admin_1
2,2,checking,0,2020-04-16 21:46:47.769088,7,None,admin_1
3,3,ready,0,2020-04-16 21:46:48.121217,7,lab02,admin_1
4,4,checking,0,2020-04-16 21:53:01.862637,6,code_rvw,admin_1
5,5,ready,0,2020-04-16 21:53:05.373389,6,code_rvw,admin_1
6,6,checking,0,2020-04-17 05:18:51.965864,1,None,None
7,7,ready,0,2020-04-17 05:19:02.744528,1,project1,user_4
8,8,checking,0,2020-04-17 05:22:35.249331,2,project1,user_4
9,9,ready,1,2020-04-17 05:22:45.549397,2,project1,user_4


Выберем строки из таблицы по условиям:

∗ status = ’ready’

∗ numTrials = 1

∗ labnames should be from the list: ’laba04’, ’laba04s’, ’laba05’, ’laba06’, ’laba06s’, ’project1’

◦ сохраним в фрейме данных контрольные элементы со столбцом cnt

In [5]:
query = '''
SELECT COUNT(*)
FROM pageviews
WHERE pageviews.uid IN
    (SELECT checker.uid
    FROM checker
    WHERE status='ready'
    AND numTrials=1
    AND labname in ('laba04', 'laba04s', 'laba05', 'laba06', 'laba06s', 'project1')
    )
'''
checkers = pd.io.sql.read_sql(query, connect)
checkers.columns = ['cnt']
checkers

,cnt
0,985


Закроем соединение с базой данных

In [6]:
connect.close()